'kinematic-corrections.py' from commit 'fead868' of https://github.com/kateharborne/kinematic_corrections is used for this code.

## how to run this example:

- git clone https://github.com/astrohchung/spin_correction_test
- enter the directory
- git submodule add https://github.com/kateharborne/kinematic_corrections
- change the permission of './kinematic_corrections/kinematic-corrections.py' file to executable.
- cd kinematic_corrections
- chmod 777 kinematic-corrections.py

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import system


In [ ]:
#correction function manually quoted from Harborne2020 (for elr, not lr)
#equation (18) and (A1) from Harborne2020
def f_por(por):
    return (7.49/(1+np.exp(4.01*por**1.57+2.84))-0.41)

def f_e_n_rfac(e,n,rfac):
    return (0.02*np.log10(e)-0.19*np.log10(n)-0.13*rfac+0.28)

def dl_corr(por, e, n, rfac):
    return (f_por(por)+por*f_e_n_rfac(e,n,rfac))

#when psf_over_re=0.45, n=1, e=1, rfac=1
psf_over_re_check=0.45
n=1
e=1
rfac=1
lr_obs_over_lr_corr=1/10**(-dl_corr(psf_over_re_check, e, n, rfac))
print(psf_over_re_check, lr_obs_over_lr_corr)


In [ ]:
def generate_h20_corr_data(ndata=50, e=1, n=1, Reff_fac=1, filename='test_input.csv'):
    ndata=50
    psf_over_re=np.arange(ndata)/ndata*0.45+0.05
    obs_lr=np.ones(ndata)
    obs_elr=np.ones(ndata)
    # e=np.ones(ndata)*0.001
    e_in=np.ones(ndata)*e
    n_in=np.ones(ndata)*n
    Reff_fac_in=np.ones(ndata)*Reff_fac
    # Re=np.ones(ndata)*1

    test_df=pd.DataFrame(
        {
            'psf_over_re':psf_over_re,
            'obs_lr':obs_lr,
            'obs_elr':obs_elr,
            'e':e_in,
            'n':n_in,
            'Reff_fac':Reff_fac_in,
    #         'Re':Re,
    #         'fwhm':fwhm,
        }
    )
    test_df.to_csv(filename)

In [ ]:
#generate n=1, e=1 data
sample_data_filename='sample_input_n1_e1.csv'
generate_h20_corr_data(ndata=50, e=1, n=1, Reff_fac=1, filename=sample_data_filename)
sample_data_corr_filename='sample_output_n1_e1.csv'

dqt='"'
cmd='./kinematic_corrections/kinematic-corrections.py'+' -i '+dqt+sample_data_filename+dqt+' -o '+dqt+sample_data_corr_filename+dqt
print(cmd)
system(cmd)

#generate n=1, e=0.01 data
sample_data_filename='sample_input_n1_e001.csv'
generate_h20_corr_data(ndata=50, e=0.01, n=1, Reff_fac=1, filename=sample_data_filename)
sample_data_corr_filename='sample_output_n1_e001.csv'

dqt='"'
cmd='./kinematic_corrections/kinematic-corrections.py'+' -i '+dqt+sample_data_filename+dqt+' -o '+dqt+sample_data_corr_filename+dqt
print(cmd)
system(cmd)


In [ ]:

df_n1_e1=pd.read_csv('sample_output_n1_e1.csv')
df_n1_e001=pd.read_csv('sample_output_n1_e001.csv')

corr_elr_n1_e1=df_n1_e1['corr_elr']
corr_elr_n1_e001=df_n1_e001['corr_elr']
psf_over_re=df_n1_e001['psf_over_re']


In [ ]:
fig=plt.figure(figsize=(6,6))
ax=fig.add_subplot(1,1,1)
ratio_n1_e1=1/corr_elr_n1_e1
ratio_n1_e001=1/corr_elr_n1_e001
ax.plot(psf_over_re, ratio_n1_e1, color='k', lw=3, linestyle='--')
ax.plot(psf_over_re, ratio_n1_e001, color='k', lw=3, linestyle='-')
ax.plot([psf_over_re_check],[lr_obs_over_lr_corr], 'bo')


xr=[0.05,0.5]
yr=[0.35,1.25]
ax.set_xlim(xr)
ax.set_ylim(yr)
ax.set_title('n_sersic=1')
ax.set_xlabel('sigma_psf / R_e^maj.')
ax.set_ylabel('lr_obs/lr_corr')

ax.tick_params(axis='both', which='both', direction='in', left=True, right=True, top=True, bottom=True)
#the difference between the dahsed line and blue dot is due to the difference in constants of correction function used. 
#The paper has two significant digits for the correction coefficient. 
#The correction function uploaded on GitHub contains more significant digits.  
#see https://github.com/kateharborne/kinematic_corrections/blob/master/kinematic-corrections.py